# apenas 12 variaveis - todos vazamentos

In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xlrd

In [38]:
variables = ["TEI","TEO","TCI","TCO","PRE","PRC","T_suc","TR_dis","Amps","kW","TO_feed","PO_feed"]
print(len(variables))

12


In [39]:
sheet = "Complete Data Set"
path_to_xls = "E:/Talles/Doutorado/database/1043-RP/FDD Data/Refrigerant leak/rl10.xls"
leaking_10 = pd.read_excel(path_to_xls,sheet)
leaking_10 = leaking_10.loc[:, variables]
path_to_xls = "E:/Talles/Doutorado/database/1043-RP/FDD Data/Refrigerant leak/rl20.xls"
leaking_20 = pd.read_excel(path_to_xls,sheet)
leaking_20 = leaking_20.loc[:, variables]
path_to_xls = "E:/Talles/Doutorado/database/1043-RP/FDD Data/Refrigerant leak/rl30.xls"
leaking_30 = pd.read_excel(path_to_xls,sheet)
leaking_30 = leaking_30.loc[:, variables]
path_to_xls = "E:/Talles/Doutorado/database/1043-RP/FDD Data/Refrigerant leak/rl40.xls"
leaking_40 = pd.read_excel(path_to_xls,sheet)
leaking_40 = leaking_40.loc[:, variables]
path_to_xls = "E:/Talles/Doutorado/database/1043-RP/FDD Data/Benchmark Tests/normal r.xls"
normal = pd.read_excel(path_to_xls,sheet)
normal = normal.loc[:, variables]
print("shapes -- leaking10 = {}, leaking 20 = {}, leaking30 = {}, leaking40 = {}, normal = {}".format(leaking_10.shape,
                                                                                                      leaking_20.shape,
                                                                                                      leaking_30.shape,
                                                                                                      leaking_40.shape,
                                                                                                      normal.shape))

shapes -- leaking10 = (5191, 12), leaking 20 = (5191, 12), leaking30 = (5191, 12), leaking40 = (5191, 12), normal = (5191, 12)


In [40]:
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler

In [41]:
# adding column with constant value
# 0 for no leak, 1 for leaking
normal['y'] = pd.Series([0 for x in range(len(normal.index))])
leaking_10['y'] = pd.Series([1 for x in range(len(leaking_10.index))])
leaking_20['y'] = pd.Series([2 for x in range(len(leaking_20.index))])
leaking_30['y'] = pd.Series([3 for x in range(len(leaking_30.index))])
leaking_40['y'] = pd.Series([4 for x in range(len(leaking_40.index))])
print("shapes -- leaking10 = {}, leaking 20 = {}, leaking30 = {}, leaking40 = {}, normal = {}".format(leaking_10.shape,
                                                                                                      leaking_20.shape,
                                                                                                      leaking_30.shape,
                                                                                                      leaking_40.shape,
                                                                                                      normal.shape))

shapes -- leaking10 = (5191, 13), leaking 20 = (5191, 13), leaking30 = (5191, 13), leaking40 = (5191, 13), normal = (5191, 13)


# Trim Edge

In [42]:
def trimEdge (dtset,n_excluded):
    dtset = dtset[n_excluded:]
    return dtset

In [43]:
# extracted 20% y for test TimeSeries
def extracted20(pdDataserie):
    bound = int(len(pdDataserie)*.8)
    train = pdDataserie[:bound]
    test = pdDataserie[bound:]
    return train, test

In [44]:
exclude = 500
normal = trimEdge(normal,exclude)
leaking_10 = trimEdge(leaking_10,exclude)
leaking_20 = trimEdge(leaking_20,exclude)
leaking_30 = trimEdge(leaking_30,exclude)
leaking_40 = trimEdge(leaking_40,exclude)

In [45]:
train_normal, test_normal = extracted20(normal)
train_leaking10, test_leaking10 = extracted20(leaking_10)
train_leaking20, test_leaking20 = extracted20(leaking_20)
train_leaking30, test_leaking30 = extracted20(leaking_30)
train_leaking40, test_leaking40 = extracted20(leaking_40)

In [46]:
# concatenating train dataframes
data_train = train_normal.append(train_leaking10, ignore_index=True, sort=False)
data_train = data_train.append(train_leaking20, ignore_index=True, sort=False)
data_train = data_train.append(train_leaking30, ignore_index=True, sort=False)
data_train = data_train.append(train_leaking40, ignore_index=True, sort=False)
print(data_train.shape)

data_train = np.asarray(data_train)
x_train,y_train = (data_train[:,:-1],data_train[:,-1])
y_train = np.expand_dims(y_train, axis=1)
print(x_train.shape,y_train.shape)

# concatenating test dataframes
data_test = test_normal.append(test_leaking10, ignore_index=True, sort=False)
data_test = data_test.append(test_leaking20, ignore_index=True, sort=False)
data_test = data_test.append(test_leaking30, ignore_index=True, sort=False)
data_test = data_test.append(test_leaking40, ignore_index=True, sort=False)
print(data_test.shape)

data_test = np.asarray(data_test)
x_test,y_test = (data_test[:,:-1],data_test[:,-1])
y_test = np.expand_dims(y_test, axis=1)
print(x_test.shape,y_test.shape)

(18760, 13)
(18760, 12) (18760, 1)
(4695, 13)
(4695, 12) (4695, 1)


In [47]:
main_path = '../overfitting/setTraining/select_noOhe/TimeSeries_Prepared/y_extracted/'
np.save(main_path +'train.npy',x_train)
np.save(main_path +'y_train.npy',y_train)
np.save(main_path +'test.npy',x_test)
np.save(main_path +'y_test.npy',y_test)